In [1]:
import os
%pwd

'c:\\Deepankar\\Projects\\Text-Summarizer\\research'

In [2]:
os.chdir('../')
%pwd

'c:\\Deepankar\\Projects\\Text-Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path


# entity 
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int 



In [4]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [5]:
# 4. Update the configuration manager in src config

from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories
from TextSummarizer.entity import DataIngestionConfig, DataValidationConfig, DataTransformationConfig


class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH,            
            ) -> None:
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config= self.config.model_trainer
        params= self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config= ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            # num_train_epochs= config.num_train_epochs,
            num_train_epochs= params.num_train_epochs,
            # warmup_steps= config.warmup_steps,
            warmup_steps= params.warmup_steps,
            # per_device_train_batch_size= config.per_device_train_batch_size,
            per_device_train_batch_size= params.per_device_train_batch_size,
            # weight_decay= config.weight_decay,
            weight_decay= params.weight_decay,
            # logging_steps= config.logging_steps,
            logging_steps= params.logging_steps,
            # evaluation_strategy= config.evaluation_strategy,
            evaluation_strategy= params.evaluation_strategy,
            # eval_steps= config.eval_steps,
            eval_steps= params.eval_steps,
            # save_steps= config.save_steps,
            save_steps= params.save_steps,
            # gradient_accumulation_steps= config.gradient_accumulation_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps,

        )


        return model_trainer_config

In [6]:
import torch

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
torch.cuda.empty_cache()

In [8]:
# ! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [9]:
class ModelTrainer:
    def __init__(self, model_trainer_config: ModelTrainerConfig) -> None:
        self.config= model_trainer_config

    def train(self):
        device= 'cuda' if torch.cuda.is_available() else 'cpu'
        # device= 'cpu'
        torch.cuda.empty_cache()
        tokenizer= AutoTokenizer.from_pretrained(self.config.model_ckpt)
        torch.cuda.empty_cache()
        # model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        torch.cuda.empty_cache()
        seq2seq_data_collator= DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


        # loading data
        dataset_samsum_pt= load_from_disk(self.config.data_path)

        print(f'save steps= {self.config.save_steps}')
        trainer_args= TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            # learning_rate=self.config.learning_rate,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps, 
            gradient_accumulation_steps= self.config.gradient_accumulation_steps
        )


        trainer= Trainer(
            model=model_pegasus,
            args=trainer_args,
            data_collator=seq2seq_data_collator,
            tokenizer=tokenizer,
            train_dataset=dataset_samsum_pt['test'],
            eval_dataset=dataset_samsum_pt['validation']
        )

        torch.cuda.empty_cache()
        trainer.train()

        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
                

In [10]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [11]:
# torch.cuda.empty_cache()

In [12]:
# torch.cuda.memory_stats()

In [13]:
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
# os.environ["PYTORCH_NO_CUDA_MEMORY_CACHING"] = "1"

In [14]:
try: 
    config= ConfigurationManager()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer= ModelTrainer(model_trainer_config= model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-07-20 01:12:28,992: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-20 01:12:28,992: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-20 01:12:28,992: INFO: common: created directory at: artifacts]
[2023-07-20 01:12:29,000: INFO: common: created directory at: artifacts/model_trainer]
save steps= 1000000


C:\Users\deepa\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/51 [00:00<?, ?it/s]

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 376.00 MiB (GPU 0; 6.00 GiB total capacity; 5.08 GiB already allocated; 0 bytes free; 5.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# import torch
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq

# torch.cuda.empty_cache()

# # model_pegasus= AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-cnn_dailymail').to('cuda')
# # model_pegasus= AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail").to('cuda')
# device= 'cuda' if torch.cuda.is_available() else 'cpu'
# tokenizer= AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
# model_pegasus= AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail").to(device)
# seq2seq_data_collator= DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


In [ ]:
# torch.cuda.memory_summary()